# Consolidação de Arquivos - Detalhes

In [1]:
# "latin1", "iso-8859-1", ou "cp1252",

import re
import os
from pathlib import Path

import duckdb
import openpyxl
import pandas as pd
from pandas import DataFrame

In [2]:
def _importar_arquivo(arquivo: str, aba: str=None, encoding: str ="latin1") -> DataFrame:
    try:
        if arquivo.endswith(".xlsx") or arquivo.endswith(".xls"):
            df = pd.read_excel(arquivo, engine="openpyxl", sheet_name=aba, dtype="str")
        elif arquivo.endswith(".csv") or arquivo.endswith(".txt"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".json"):
            df = pd.read_json(arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado")
        return df
    except Exception as erro:
        raise ValueError(f"Erro: {erro}")

In [3]:
def amostra_atributo(atributo: str, dataframe: DataFrame, num_amostras:int=2) -> str:
    try:
        query = f"""
        SELECT {atributo}
        FROM dataframe WHERE {atributo}
        IS NOT NULL
        LIMIT {num_amostras}
        """
        resultado_query = (
            [
                str(item[0])
                .replace(",", "")
                .strip()
                for item
                in duckdb
                .sql(query)
                .fetchall()
            ]
        )
        if len(resultado_query) == 0:
            resultado_query = ["SEM AMOSTRA"]
        return ";".join(resultado_query)
    except Exception as _:
        return "SEM AMOSTRA" 

In [4]:
def detalhes_arquivos(lista_arquivos: list) -> DataFrame:
    try:
        lista_temp = []
        for item in lista_arquivos:
            caminho = Path(os.path.relpath(item)).parent
            arquivo = f"{Path(os.path.abspath(item)).stem}{Path(os.path.abspath(item)).suffix}" 
            try:
                if item.endswith(".xlsx") or item.endswith(".xls"): 
                    abas_planilha = pd.ExcelFile(item).sheet_names

                    for aba in abas_planilha:
                        df = _importar_arquivo(item, aba)
                        colunas = [coluna for coluna in df.columns.tolist()]
                        quantidades = df.count().tolist()

                        for coluna, quantidade in zip(colunas, quantidades):
                            amostra = amostra_atributo(coluna, df)
                            lista_temp.append(
                                {   
                                    "caminho": caminho,
                                    "arquivo": arquivo,
                                    "coluna": coluna.strip(),
                                    "amostra": amostra,
                                    "quantidade": quantidade,
                                    "aba": aba,
                                    "observacoes": "",
                                    "status": "OK",
                                    "nova_coluna": (
                                        "***Esse atributo pode renomear "
                                        "as colunas na consolidação final.*** "
                                    )
                                    }
                            )
                else:
                    df = _importar_arquivo(item)
                    colunas = [coluna for coluna in df.columns.tolist()]
                    quantidades = df.count().tolist()
                    for coluna, quantidade in zip(colunas, quantidades):
                        amostra = amostra_atributo(coluna, df)
                        lista_temp.append(
                            {   
                                "caminho": caminho,
                                "arquivo": arquivo,
                                "coluna": coluna.strip(),
                                "amostra": amostra,
                                "quantidade": quantidade,
                                "aba": "",
                                "observacoes": "",
                                "status": "OK",
                                "nova_coluna": (
                                    "***Esse atributo pode renomear "
                                    "as colunas na consolidação final.*** "
                                )
                                }
                        )
            except Exception as erro:
                lista_temp.append(
                    {
                        "caminho": caminho,
                        "arquivo": arquivo,
                        "coluna": "",
                        "amostra": "",
                        "quantidade": "",
                        "aba": "",                        
                        "observacoes": f"{erro}",
                        "status": "ERRO",
                        "nova_coluna": ""
                        }
                )
        return pd.DataFrame(lista_temp)
    except Exception as erro:
        raise ValueError(f"Erro na geração dos detalhes. Erro: {erro}")

### Importando a Lista de arquivos a partir dos Metadados

In [5]:
metadados_arquivos = pd.read_csv("../resultados/metadados/consolidacao_metadados.csv", sep=";", dtype="str")

In [6]:
metadados_arquivos

,caminho,arquivo,extensao,modo,inode,dispositivo,links,uid,gid,tamanho,ultimo_acesso,ultima_modificacao,criacao
0,../arquivos,informacao-empresas.csv,csv,33204,13799852,66307,1,1001,1001,84,10/07/2024 03:53:04,05/07/2024 04:40:28,05/07/2024 04:44:54
1,../arquivos/dir2/dir2.1,informacao-salario.csv,csv,33204,13799853,66307,1,1001,1001,69,10/07/2024 03:53:04,06/07/2024 17:22:59,06/07/2024 17:22:59
2,../arquivos,informacao-erro.jpg,jpg,33204,15230954,66307,1,1001,1001,0,06/07/2024 17:29:31,06/07/2024 17:29:31,06/07/2024 17:29:31
3,../arquivos/dir2,informacao-endereco.json,json,33204,13799848,66307,1,1001,1001,228,10/07/2024 03:53:04,07/07/2024 05:13:20,07/07/2024 05:13:20
4,../arquivos/dir1,informacao-pessoa.json,json,33204,13799849,66307,1,1001,1001,174,10/07/2024 03:53:04,05/07/2024 04:40:22,06/07/2024 16:18:56
5,../arquivos/dir1,informacao-erro.png,png,33204,15226559,66307,1,1001,1001,0,06/07/2024 17:29:13,06/07/2024 17:29:12,06/07/2024 17:29:12
6,../arquivos,informacao-cargo.txt,txt,33204,13799851,66307,1,1001,1001,77,10/07/2024 03:53:04,05/07/2024 04:40:26,05/07/2024 04:46:46
7,../arquivos/dir2,informacoes-contatos.txt,txt,33204,13799850,66307,1,1001,1001,144,10/07/2024 03:53:04,05/07/2024 04:40:24,06/07/2024 16:18:54
8,../arquivos/dir1,informacao-departamento.xlsx,xlsx,33204,13799854,66307,1,1001,1001,5517,10/07/2024 03:53:04,05/07/2024 04:40:32,06/07/2024 16:17:45
9,../arquivos,informacao-projeto.xlsx,xlsx,33204,15230982,66307,1,1001,1001,6704,10/07/2024 03:53:04,07/07/2024 04:35:56,07/07/2024 04:35:56


In [7]:
lista_arquivos = (
    [
        f"{caminho}/{arquivo}"
        for caminho, arquivo
        in zip(metadados_arquivos["caminho"], metadados_arquivos["arquivo"])
    ]
)

In [8]:
lista_arquivos

['../arquivos/informacao-empresas.csv',
 '../arquivos/dir2/dir2.1/informacao-salario.csv',
 '../arquivos/informacao-erro.jpg',
 '../arquivos/dir2/informacao-endereco.json',
 '../arquivos/dir1/informacao-pessoa.json',
 '../arquivos/dir1/informacao-erro.png',
 '../arquivos/informacao-cargo.txt',
 '../arquivos/dir2/informacoes-contatos.txt',
 '../arquivos/dir1/informacao-departamento.xlsx',
 '../arquivos/informacao-projeto.xlsx']

In [9]:
df_detalhes_arquivos = detalhes_arquivos(lista_arquivos)

In [10]:
df_detalhes_arquivos.sort_values(by=["arquivo","aba","coluna"])

,caminho,arquivo,coluna,amostra,quantidade,aba,observacoes,status,nova_coluna
22,../arquivos,informacao-cargo.txt,Cargo,Gerente;Analista,3,,,OK,***Esse atributo pode renomear as colunas na c...
19,../arquivos,informacao-cargo.txt,ID,1;2,3,,,OK,***Esse atributo pode renomear as colunas na c...
21,../arquivos,informacao-cargo.txt,Idade,30;25,3,,,OK,***Esse atributo pode renomear as colunas na c...
20,../arquivos,informacao-cargo.txt,Nome,Ana;Bruno,3,,,OK,***Esse atributo pode renomear as colunas na c...
31,../arquivos/dir1,informacao-departamento.xlsx,Departamento,RH;TI,3,Sheet1,,OK,***Esse atributo pode renomear as colunas na c...
28,../arquivos/dir1,informacao-departamento.xlsx,ID,1;2,3,Sheet1,,OK,***Esse atributo pode renomear as colunas na c...
30,../arquivos/dir1,informacao-departamento.xlsx,Idade,30;25,3,Sheet1,,OK,***Esse atributo pode renomear as colunas na c...
29,../arquivos/dir1,informacao-departamento.xlsx,Nome,Ana;Bruno,3,Sheet1,,OK,***Esse atributo pode renomear as colunas na c...
3,../arquivos,informacao-empresas.csv,Empresa,Empresa A;Empresa B,3,,,OK,***Esse atributo pode renomear as colunas na c...
0,../arquivos,informacao-empresas.csv,ID,1;2,3,,,OK,***Esse atributo pode renomear as colunas na c...


In [12]:
(
    df_detalhes_arquivos.sort_values(by=["caminho","arquivo","aba","coluna"])
    .to_csv("../resultados/detalhes/consolidacao_detalhes.csv", index=False, sep=";")
)